In [24]:
print("hello")

hello


In [25]:
print("hi")

hi


In [36]:
afjff= 1
print(afjff)

1


In [37]:
print(afjff)

1


In [45]:
a=1
b=2.0
c=a+b
print(c)

3.0


In [48]:
if a >= 1:
    print('bad')
if a != 1:
    print('good')

bad


In [52]:
for i in range(1,6,2):
    print(i)

1
3
5


In [55]:
counter =0
while counter != 2:
    print(counter)
    counter+=1

0
1


In [57]:
first = 0
second = 1
print(first)
print(second)
for i in range(10):
    third =first+second
    print(third)
    first= second
    second = third

0
1
1
2
3
5
8
13
21
34
55
89
